# Chapter 13: Operator Overloading: Doing It Right

There are different kinds of operators in Python:

* `+` and `|` are examples of **infix operators**.

* `-` and `~` are examples of **unary operators**.

* `()` (function invocation), `.` (attribute access) and `[]` (item access/slicing) are also examples of operators.

This chapter deals mainly with infix and unary operators, and will cover:

* How Python supports infix operators with operands of different types.

* Using duck typing or explicit type checks to deal with operands of various types.

* How an infix operator method should signal it cannot handle an operand.

* The special behaviour of the rich comparison operators (e.g., `==`, `>`, `<`, `<=`, etc.).

* The default handling of augmented assignment operators, like `+=`, and how to overload them.

## Operator Overloading 101

TODO.